In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# census_bureau_international, find gender specific growth rate, 
# Compare above with total population growth rate per year from "birth_death_growth_rates"
# whether growth rate was correlated to life expectancy change.
# try to calculate yearwise mortality rate.

In [ ]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

### 1. Gender-wise population growth rate of India
In India there were (are) various socio-cultural taboos associated with girl-children birth, specially in rural areas. Hence, based on the country wise data of an international census bureau (found in bigquery public dataset, accessed through Kaggle) we investigate if that has any imprint in the female population growth rate of India.  The following code also fills the gap in the data-set not enlisting year to year genderwise population growth rate of a country. We also compare the female growth rate of India with global average.

In [ ]:
# The SQL code accesses two tables in the dataset:
# "midyear_population_agespecific" table: Used columns- country_code, sex, year, population (for midyear population)
# "birth_death_growth_rates" table: We use columns- country_code, year, growth_rate

# A combination of many SQL concepts are used: Aggregate (window) functions including conditional aggregation, Joining tables, 
# CTE, GROUP BY, ORDER BY and so on.

query4 = """WITH country_yearwise_tot_pop AS
            (SELECT country_code, sex, year, SUM(population) AS tot_population
            from `bigquery-public-data.census_bureau_international.midyear_population_agespecific`
            GROUP BY sex,year,country_code
            ),
            yearwise_pop AS
            (SELECT sex,year,SUM(tot_population) AS glob_pop,
            SUM(CASE WHEN country_code ='IN' THEN tot_population ELSE 0 END) AS ind_pop
            from country_yearwise_tot_pop
            GROUP BY sex, year
            )
            SELECT y.year, y.ind_pop as IND_total_female_population,
            (y.glob_pop-lag(y.glob_pop)
                OVER(PARTITION BY y.sex
                    ORDER BY y.year))*100/(lag(y.glob_pop)
                    OVER(PARTITION BY y.sex
                    ORDER BY y.year)) AS glob_female_pop_growth_rate,
            (y.ind_pop-lag(y.ind_pop)
                OVER(PARTITION BY y.sex
                    ORDER BY y.year))*100/(lag(y.ind_pop)
                    OVER(PARTITION BY y.sex
                    ORDER BY y.year)) AS IND_female_pop_growth_rate, bg.growth_rate AS IND_tot_pop_growth_rate
            from yearwise_pop AS y
            INNER JOIN `bigquery-public-data.census_bureau_international.birth_death_growth_rates` AS bg
                ON y.year=bg.year
            WHERE bg.country_code='IN' AND y.sex='Female'
            ORDER BY IND_female_pop_growth_rate DESC
            """

safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**4)
query_job = client.query(query4, job_config=safe_config)
result4 = query_job.result().to_dataframe()

In [ ]:
### First few lines of the output table

result4.iloc[0:10,:]

In [ ]:
ax1 = result4.plot(kind='scatter',x="year", y="glob_female_pop_growth_rate",color='b',label='glob_female_pop_growth_rate',alpha=0.5)  
ax2 = result4.plot(kind='scatter', x="year", y="IND_female_pop_growth_rate",color='g',label='IND_female_pop_growth_rate',alpha=0.5, ax=ax1)
ax3 = result4.plot(kind='scatter', x="year", y="IND_tot_pop_growth_rate",color='r',label='IND_tot_pop_growth_rate',alpha=0.5, ax=ax1)

ax1.set_ylabel(' ')